In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
from skimage.measure import shannon_entropy

In [2]:
def data_load(npypath, crop_size = 768):
#npypath = '/data/asim/ISP/HDR_transformer/data/RAW/2022-0708-1630-2434.npz'
    imdata = np.load(npypath)

    sht = imdata['sht']
    mid = imdata['mid']
    lng = imdata['lng']
    hdr = imdata['hdr']


    H, W = hdr.shape[1], hdr.shape[2]
    start_h = (H - crop_size) // 2
    start_w = (W - crop_size) // 2

    sht_crop = sht[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
    mid_crop = mid[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
    lng_crop = lng[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
    hdr_crop = hdr[:, start_h:start_h+crop_size, start_w:start_w+crop_size]
    return sht_crop, mid_crop

In [3]:

def bayer_to_rgb(bayer_array):
    H, W = bayer_array.shape[1:]
    bayer = np.zeros((H * 2, W * 2), dtype=bayer_array.dtype)
    bayer[0::2, 0::2] = bayer_array[0]
    bayer[0::2, 1::2] = bayer_array[1]
    bayer[1::2, 0::2] = bayer_array[2]
    bayer[1::2, 1::2] = bayer_array[3]

    bayer = np.clip(bayer, 0, 1)
    bayer = (bayer * 65535).astype(np.uint16)

    #print("Max value before demosaicing:", np.max(bayer))

    rgb = cv2.cvtColor(bayer, cv2.COLOR_BayerBG2RGB)

    rgb = rgb.astype(np.float32) / 65535.0

    #print("Max RGB value after demosaicing:", np.max(rgb))

    # --- Normalize to maximum brightness ---
    if np.max(rgb) > 0:
        rgb = rgb / np.max(rgb)

    # Optional: Apply gamma correction (makes image bright and natural)
    rgb = np.clip(rgb, 0, 1) ** (1/2.2)

    return rgb



In [4]:
npypath1 = '/data/asim/ISP/HDR_transformer/data/RAW/2022-0708-1630-2434.npz'
npypath2 = '/data/asim/ISP/HDR_transformer/data/RAW/2022-0703-2242-0173.npz'
sht1, mid1 = data_load(npypath1)
sht2, mid2 = data_load(npypath2)

In [5]:
sht_rgb_1 = bayer_to_rgb(sht1[:4, :, :])
mid_rgb_1 = bayer_to_rgb(mid1[:4, :, :])
#plt.imshow(im1_rgb)
sht_rgb_2 = bayer_to_rgb(sht2[:4, :, :])
mid_rgb_2 = bayer_to_rgb(mid2[:4, :, :])

In [6]:
'''overexposed_mask_in_mid = np.mean(im2_rgb, axis =2)>0.9
fixed_im2_rgb = im2_rgb.copy()
fixed_im2_rgb[overexposed_mask_in_mid]= im1_rgb[overexposed_mask_in_mid]
plt.imshow(fixed_im2_rgb)'''

'overexposed_mask_in_mid = np.mean(im2_rgb, axis =2)>0.9\nfixed_im2_rgb = im2_rgb.copy()\nfixed_im2_rgb[overexposed_mask_in_mid]= im1_rgb[overexposed_mask_in_mid]\nplt.imshow(fixed_im2_rgb)'

In [7]:

def compute_entropy(img):
    if len(img.shape) == 3:  # RGB
        return np.mean([shannon_entropy(img[:, :, c]) for c in range(3)])
    return shannon_entropy(img)  # Grayscale

# Compute entropy
H_mid1 = compute_entropy(mid1)
H_sht1 = compute_entropy(sht1)
H_mid2 = compute_entropy(mid2)
H_sht2 = compute_entropy(sht2)
print(H_mid1, H_mid2)
# Exposure variance for each sample
exposure_var_A = abs(H_mid1 - H_sht1)
exposure_var_B = abs(H_mid2 - H_sht2)

# Difference in exposure variances
delta = abs(exposure_var_A - exposure_var_B)

print("Sample A Exposure Variance:", exposure_var_A)
print("Sample B Exposure Variance:", exposure_var_B)
#print("Variance Difference (delta):", delta)


7.364266241495598 7.257632017813832
Sample A Exposure Variance: 0.7761919560332151
Sample B Exposure Variance: 0.7814297747634624
